##### Author: Nisha Sundaram

In [ ]:
# importing the necessary libraries

from bs4 import BeautifulSoup, SoupStrainer #HTML parsing
import urllib.request #calling the URLs
from time import sleep #with that we put the scraper to sleep
import json #reading and writing JSON files
from datetime import datetime #to timestamp the data
import re #regular expressions
import os #to create and read file paths
import pandas as pd #for data analysis and manipulation

folders = ["data/visited_20/","data/autos_20/"]

for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)
        print(path, "erstellt.")
    else:
        print(folder,"existiert bereits")

path_to_visited_urls = "data/visited/visited_urls.json"

if not os.path.isfile(path_to_visited_urls):
    with open(path_to_visited_urls,"w") as file:
        json.dump([],file)

# countries = { "Germany" : "D"; "Austria" : "A", "Belgium" : "B"; "Spain" : "s"; "France" : "F"; "Italy" : "I"; "Luxembourg" : "L"; "Netherlands" : "NL" }

# picking only Germany
countries = {"Deutschland": "D"}

# choosing only the TOP 10 branded cars
car_brand_list = {"mercedes-benz","volkswagen","bmw","audi","ford","opel","skoda","porsche","toyota","renault"}

# creating a URL base
url_base = "https://www.autoscout24.de/lst/"

url_search = ["?fregfrom=2020&fregto=2020&sort=standard&desc=0&cy=D&atype=C&ustate=N%2CU&powertype=kw&pricefrom=1000&priceto=10000&ocs_listing=include",
"?fregfrom=2020&fregto=2020&sort=standard&desc=0&cy=D&atype=C&ustate=N%2CU&powertype=kw&pricefrom=10001&priceto=20000&ocs_listing=include",
"?fregfrom=2020&fregto=2020&sort=standard&desc=0&cy=D&atype=C&ustate=N%2CU&powertype=kw&pricefrom=20001&priceto=30000&ocs_listing=include",
"?fregfrom=2020&fregto=2020&sort=standard&desc=0&cy=D&atype=C&ustate=N%2CU&powertype=kw&pricefrom=30001&priceto=40000&ocs_listing=include",
"?fregfrom=2020&fregto=2020&sort=standard&desc=0&cy=D&atype=C&ustate=N%2CU&powertype=kw&pricefrom=40001&priceto=50000&ocs_listing=include",
"?fregfrom=2020&fregto=2020&sort=standard&desc=0&cy=D&atype=C&ustate=N%2CU&powertype=kw&pricefrom=50001&priceto=60000&ocs_listing=include",
"?fregfrom=2020&fregto=2020&sort=standard&desc=0&cy=D&atype=C&ustate=N%2CU&powertype=kw&pricefrom=60001&priceto=70000&ocs_listing=include",
"?fregfrom=2020&fregto=2020&sort=standard&desc=0&cy=D&atype=C&ustate=N%2CU&powertype=kw&pricefrom=70001&priceto=80000&ocs_listing=include",
"?fregfrom=2020&fregto=2020&sort=standard&desc=0&cy=D&atype=C&ustate=N%2CU&powertype=kw&pricefrom=80001&priceto=90000&ocs_listing=include",
"?fregfrom=2020&fregto=2020&sort=standard&desc=0&cy=D&atype=C&ustate=N%2CU&powertype=kw&pricefrom=90001&priceto=100000&ocs_listing=include"]

car_counter=1
cycle_counter=0

while True:

    with open(path_to_visited_urls) as file:
        visited_urls = json.load(file)
    
    if len(visited_urls) > 100000:
        visited_urls = []
    
    multiple_cars_dict = {}
    
    cycle_counter+=1

    #for country in countries:
    for car_brand in car_brand_list:
        
        car_URLs = []
        
        for page in range(1,21):

            try:
                #url = 'https://www.autoscout24.de/lst/?sort=age&desc=1&ustate=N%2CU&size=20&page='+str(page)+ '&cy=' + countries[country] +'&atype=C&'                
                url = url_base + car_brand + url_search[(cycle_counter-1)%10]+'&page='+str(page);
                #print(url)
                #url = 'https://www.autoscout24.de/lst/mercedes-benz?sort=age&desc=1&ustate=N%2CU&size=20&page='+str(page)+ '&cy=' + countries[country] +'&atype=C&'
                only_a_tags = SoupStrainer("a")
                soup = BeautifulSoup(urllib.request.urlopen(url).read(),'lxml', parse_only=only_a_tags)
            except Exception as e:
                print("Übersicht: " + str(e) +" "*50, end="\r")
                pass

            for link in soup.find_all("a"):
                if r"/angebote/" in str(link.get("href")):
                    car_URLs.append(link.get("href"))
                    
            #print(car_URLs)
            car_URLs_unique = [car for car in list(set(car_URLs)) if car not in visited_urls]
            
            print(f'Lauf {cycle_counter} | {car_brand} | Seite {page} | {len(car_URLs_unique)} neue URLs', end="\r")
        print("")
        if len(car_URLs_unique)>0:

            for URL in car_URLs_unique:
                print(f'Lauf {cycle_counter} | {car_brand} | Auto {car_counter}'+' '*50, end="\r")
                try:
                    car_counter+=1

                    car_dict = {}
                    car_dict["car_brand"] = car_brand
                    car_dict["date"] = str(datetime.now())                    
                    car = BeautifulSoup(urllib.request.urlopen('https://www.autoscout24.de'+URL).read(),'lxml')
                    #print("https://www.autoscout24.de"+URL)
                    #print(car)
                    #print("I am here 1")
                    #print(car_dict)
                    
                    for key, value in zip(car.find_all("dt"),car.find_all("dd")):
                        car_dict[key.text.replace("\n","")] = value.text.replace("\n","")

                    #print("I am here 2")
                    #print(car_dict)
                    car_dict["verkaeufer"] = car.select("#__next > div > main > div.css-11g4f57 > div.css-cc01rj > div > div.css-1daeinn > div.css-fm173v > span")[0].get_text()
                    car_dict["brand"] = car.select("#__next > div > main > div.css-157je49 > div.css-11y8sgd > div.css-12ipdh9 > div.css-1m1xare > h1 > div.css-11siofd.errr7t01 > span.css-a36h5")[0].get_text()
                    car_dict["model"] = car.select("#__next > div > main > div.css-157je49 > div.css-11y8sgd > div.css-12ipdh9 > div.css-1m1xare > h1 > div.css-11siofd.errr7t01 > span.css-1mhe8d.errr7t00")[0].get_text()
                    car_dict["model_details"] = car.select("#__next > div > main > div.css-157je49 > div.css-11y8sgd > div.css-12ipdh9 > div.css-1m1xare > h1 > div.css-l08njs")[0].get_text()
                    
                    #car_dict["haendler"] = car.find("div",attrs={"class":"cldt-vendor-contact-box",
                    #                                             "data-vendor-type":"dealer"}) != None
                    #car_dict["privat"] = car.find("div",attrs={"class":"cldt-vendor-contact-box",
                    #                                           "data-vendor-type":"privateseller"}) != None

                    #print(car_dict)
                    #print("I am here 4")
                    #ort_val = car.find("div",attrs={"class":"sc-grid-col-12","data-item-name":"vendor-contact-city"})
                    #print(ort_val)
                    #car_dict["ort"] = car.find("div",attrs={"class":"sc-grid-col-12",
                    #                                        "data-item-name":"vendor-contact-city"})
                    
                    price_val = car.select("#__next > div > main > div.css-157je49 > div.css-11y8sgd > div.css-1371dt3 > div:nth-child(1) > div > div.css-s5xdrg > span")[0].get_text()
                    #print(price_val)
                    x = price_val.split(",")
                    #print(x[0])                    
                    car_dict["price"] =  "".join(re.findall(r'[0-9]+',x[0]))
                    
                    #ausstattung = []
                    
                    #for i in car.find_all("div",attrs={"class":"cldt-equipment-block sc-grid-col-3 sc-grid-col-m-4 sc-grid-col-s-12 sc-pull-left"}):
                    #    for span in i.find_all("span"):
                    #        ausstattung.append(i.text)

                    #ausstattung2 = []

                    #for element in list(set(ausstattung)):
                    #    austattung_liste = element.split("\n")
                    #    ausstattung2.extend(austattung_liste)

                    #car_dict["ausstattung_liste"] = sorted(list(set(ausstattung2)))
                    #print("I am here 5")
                    #print(car_dict)

                    multiple_cars_dict[URL] = car_dict
                    visited_urls.append(URL)
                except Exception as e:
                    print("Detailseite: " + str(e) + " "*50)
                    pass
            print("")
        
        else:
            print("\U0001F634")
            sleep(1)
    
    if len(multiple_cars_dict)>0:
        df = pd.DataFrame(multiple_cars_dict).T
        df.to_csv("data/autos/"+re.sub("[.,:,-, ]","_",str(datetime.now()))+".csv",sep=";",index_label="url")
    else:
        print("Keine Daten")
    with open("data/visited/visited_urls.json", "w") as file:
        json.dump(visited_urls, file)

data/visited_20/ existiert bereits
data/autos_20/ existiert bereits
Lauf 1 | volkswagen | Seite 20 | 5 neue URLs
Detailseite: list index out of range                                                  
Lauf 1 | volkswagen | Auto 5                                                  
Lauf 1 | audi | Seite 20 | 0 neue URLs
😴
Lauf 1 | opel | Seite 20 | 1 neue URLs
Lauf 1 | opel | Auto 6                                                  
Lauf 1 | toyota | Seite 20 | 2 neue URLs
Lauf 1 | toyota | Auto 8                                                  
Lauf 1 | porsche | Seite 20 | 0 neue URLs
😴
Lauf 1 | skoda | Seite 20 | 0 neue URLs connection was forcibly closed by the remote host                                                  
😴
Lauf 1 | mercedes-benz | Seite 20 | 1 neue URLs
Lauf 1 | mercedes-benz | Auto 9                                                  
Lauf 1 | renault | Seite 20 | 9 neue URLs
Lauf 1 | renault | Auto 18                                                  
Lauf 1 | ford | S